In [3]:
# Construction of dataset

import os, itertools, time, pickle, sys
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline  


USE_folder = "/home/vlead/USE"
alignment_folder = "reference-alignment/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments
        
reference_alignments = load_alignments(alignment_folder)

In [4]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()
    
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        return [(b,a,"subclass_of") for (a,b) in self.get_subclasses()]
    
    def parse_triples(self, union_flag=0, subclass_of=True):
        obj_props = self.object_properties
        data_props = self.data_properties
        props = obj_props + data_props
        all_triples = []
        for prop in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop)) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop)))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True, include_inv=True):
        return self.parse_triples(union_flag, subclass_of)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    continue
                if self.extract_ID(level1_class[0]):
                    subclass_pairs.append((level1_class[0], el.parentNode))
                else:
                    level2classes = level1_class[0].getElementsByTagName("owl:Class")
                    
                    subclass_pairs.extend([(elem, el.parentNode) for elem in level2classes if self.extract_ID(elem)])
        return subclass_pairs
        
    def get_subclasses(self):
        return [(self.extract_ID(a), self.extract_ID(b)) for (a,b) in self.subclasses]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        return element_id.split("#")[-1]
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))




In [49]:
# Extracting USE embeddings

ontologies_in_alignment = [l.split(".")[0].split("-") for l in os.listdir("reference-alignment/")]

def extractUSEEmbeddings(words):
    try:
        embed = hub.KerasLayer(USE_folder)
    except Exception as e:
        !mkdir $USE_folder
        !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC $USE_folder
        embed = hub.KerasLayer(USE_folder)
        pass
    word_embeddings = embed(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0).lower() for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))

inp = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\S+")
X = vectorizer.fit_transform(inp)
word2idx_tfidf = {word: i for (i, word)  in enumerate(vectorizer.get_feature_names())}
entity2idx_tfidf = {word.split("#")[1]: i for (i, word)  in enumerate(extracted_elems)}


print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

inp = ["<UNK>"] + inp
extracted_elems = ["<UNK>"] + extracted_elems

embeds = extractUSEEmbeddings(inp)
embeddings = dict(zip(extracted_elems, embeds))    

Total number of extracted unique classes and properties from entire RA set:  834


In [50]:
# Type storage

types_dict = {}

def get_tfidf_score(word, phrase):
    return np.sum([X[entity2idx_tfidf[phrase]][:,word2idx_tfidf[word]][0,0] for word in parse(phrase)])
    
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()

    for entity in entities:
        types_dict[entity] = {"type": "entity"}
    for prop in props:
        types_dict[prop] = {"type": "property"}


In [51]:
# Combinatorial mapping generation

all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology("conference_ontologies/" + l[0] + ".owl")
    ont2 = Ontology("conference_ontologies/" + l[1] + ".owl")
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()
    
    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    all_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in mappings])
    

In [52]:
gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]

data = {}
for mapping in all_mappings:
    if mapping in gt_mappings:
        data[(mapping[0], mapping[1])] = True
    else:
        data[(mapping[0], mapping[1])] = False


In [53]:

def greedy_matching():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics
    all_results = OrderedDict()
    with torch.no_grad():
        all_pred = []
        batch_size = min(batch_size, len(test_data_t))
        num_batches = int(ceil(len(test_data_t)/batch_size))
        batch_size_f = int(ceil(len(test_data_f)/num_batches))
        
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            batch_start_f = batch_idx * batch_size_f
            batch_end_f = (batch_idx+1) * batch_size_f
            
            
            
            pos_elems = np.array(test_data_t)[batch_start:batch_end]
            neg_elems = np.array(test_data_f)[batch_start_f:batch_end_f]
            optimizer.zero_grad()

            inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])
            
            targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])
            
            print (inputs)
            
            indices = np.random.permutation(inputs.shape[0])
            inputs, targets = inputs[indices].transpose(1,0,2), targets[indices]
#             inputs = inputs.transpose(1,0,2)
            inputs_elem = inputs.copy()
            
#             print ("1", inputs_elem)
            
            nonzero_elems = np.count_nonzero(inputs, axis=-1)
            indices = np.flip(np.argsort(nonzero_elems, axis=-1), axis=-1)
            seq_lens = np.flip(np.sort(nonzero_elems, axis=-1), axis=-1)
#             print (seq_lens)
            inputs = np.stack((inputs[0][[indices[0]]], inputs[1][[indices[1]]]), axis=0)
            
            d1 = {elem:i for i,elem in enumerate(indices[0])}
            d2 = {elem:i for i,elem in enumerate(indices[1])}
            rev_indices = np.stack(([d1[k] for k in range(inputs_elem.shape[1])], [d2[k] for k in range(inputs_elem.shape[1])]))
#             print ("2", rev_indices)
#             print ("3", indices)
            rev_indices = torch.LongTensor(rev_indices)
            inputs = torch.LongTensor(inputs)
            seq_lens = torch.LongTensor(seq_lens.copy())
            targets = torch.DoubleTensor(targets)

            outputs = model(inputs, seq_lens, rev_indices)
            outputs /= torch.sum(outputs, dim=1).view(-1, 1)
#             write (("Outputs Finally: ", str([str(s) for s in outputs])))
            outputs = [(1-el[1].item()) for el in outputs]
    
#             return
#             print ("2", inputs)
#             print ("3", seq_lens)
#             print ("4", rev_indices)
            
            targets = [True if el.item() else False for el in targets]
#             print (inputs)
            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[inputs_elem[0][idx][0]]
                ent2 = emb_indexer_inv[inputs_elem[1][idx][0]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
        all_results = OrderedDict(sorted(all_results.items(), key=lambda x: x[0], reverse=True))
        filtered_results = dict()
        
        entities_to_assign = set([el[0] for el in list(all_results.keys())])
        for pair in all_results:
            if pair[0] in entities_to_assign:
                filtered_results[pair] = all_results[pair]
                entities_to_assign.remove(pair[0])
                
        entities_to_assign = set([el[1] for el in list(all_results.keys())])
        for pair in all_results:
            if pair[1] in entities_to_assign:
                filtered_results[pair] = all_results[pair]
                entities_to_assign.remove(pair[1])        

        filtered_results = OrderedDict(sorted(filtered_results.items(), key=lambda x: x[1][0], reverse=True))
        
        optimum_metrics, opt_threshold = [-1000 for i in range(5)], -1000
        low_threshold = np.min([el[0] for el in all_results.values()]) - 0.01
        high_threshold = np.max([el[0] for el in all_results.values()]) + 0.01
        for j,threshold in enumerate(np.arange(low_threshold, high_threshold, 0.01)):
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)
            fn_list = [key for key in gt_mappings if key not in set(res) and not is_valid(test_onto, key)]
            fp_list = [elem for elem in res if not all_results[elem][1]]
            tp_list = [elem for elem in res if all_results[elem][1]]
            
            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
            
            
            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except Exception as e:
                print (e)
                continue
            print ("Threshold: ", threshold, precision, recall, f1score, f2score, f0_5score)

            if f1score > optimum_metrics[2]:
                optimum_metrics = [precision, recall, f1score, f2score, f0_5score]
                opt_threshold = threshold
        
        print ("Precision: {} Recall: {} F1-Score: {} F2-Score: {} F0.5-Score: {}".format(*optimum_metrics))
        all_metrics.append((opt_threshold, optimum_metrics))
    return all_results


dict_keys(['cmt#acceptPaper', 'edas#AcceptRating', 'iasted#Author', 'edas#MealEvent', 'confOf#Administrator', 'cmt#SubjectArea', 'edas#ReviewRating', 'cmt#setMaxPapers', 'conference#Publisher', 'iasted#is_used_by', 'iasted#need', 'iasted#Speaker_lecture', 'cmt#addProgramCommitteeMember', 'sigkdd#Abstract', 'confOf#Paper', 'confOf#Member', 'confOf#hasFirstName', 'cmt#rejectPaper', 'confOf#hasPhone', 'edas#string', 'iasted#Departure_tax', 'conference#has_an_organizing_committee', 'cmt#Meta-Review', 'iasted#Listener', 'conference#has_a_commtitee', 'iasted#prepare', 'confOf#Workshop', 'cmt#readPaper', 'edas#DiningPlace', 'cmt#ConferenceChair', 'cmt#acceptsHardcopySubmissions', 'conference#is_a_date_of_camera_ready_paper_submission', 'edas#isReviewHistoryOf', 'edas#hasCostCurrency', 'ekaw#University', 'iasted#Form', 'edas#ReviewForm', 'confOf#maxChoice', 'ekaw#Individual_Presentation', 'sigkdd#Best_Applications_Paper_Award', 'conference#Call_for_paper', 'iasted#Presenter_house', 'confOf#has

In [54]:
ind_test, inp_test1, inp_test2 = None, None, None

def write(elem):
    f = open("Logs", "a+")
    if type(elem) == list or type(elem) == tuple:
        string = str("\n".join([str(s) for s in elem]))
    else:
        string = str(elem)
    f.write("\n"+string)
    f.close()
    
inputs3, results3 = None, None

class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers):
        super().__init__() 
        
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.name_embedding = nn.Embedding(len(embeddings), self.embedding_dim)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})

        self.dropout = dropout
        
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.num_layers, batch_first=True)
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        self.layer1 = nn.Bilinear(self.hidden_dim, self.hidden_dim, 2)

    def forward(self, inputs, seq_lens, rev_indices):
        results = []
        for i in range(2):
            x = self.name_embedding(inputs[i])
#             print ("Embeddings", x)
            packed_inp = pack_padded_sequence(x, seq_lens[i].numpy(), batch_first=True)
            op, (ht, ct) = self.lstm(x)
            x = ht[2*(self.num_layers-1):].permute(1,0,2)
            x = x[rev_indices[i],:,:]
            results.append(x.reshape(-1, self.hidden_dim))
        global inputs3, results3
        results3 = results
        inputs3 = inputs
        x = self.layer1(results[0], results[1])
        return x

In [57]:

emb_indexer = {word: i for i, word in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: word for i, word in enumerate(list(embeddings.keys()))}
emb_vals = list(embeddings.values())


def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology("conference_ontologies/" + ont + ".owl")
    triples = ont_obj.get_triples()
    entities = [(a,b) for (a,b,c) in triples]
    neighbours_dict = {elem: [elem] for elem in list(set(flatten(entities)))}
    for e1, e2 in entities:
        neighbours_dict[e1].append(e2)
        neighbours_dict[e2].append(e1)
    
    prop_triples = ont_obj.get_triples(subclass_of=False)
    neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
    for e1, e2, p in prop_triples:
        neighbours_dict_props[p].extend([e1, e2])

    neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
        if elem not in neighbours_dict:
            neighbours_dict[elem] = [elem]

    neighbours_dict = {el: neighbours_dict[el][:1] + sorted(list(set(neighbours_dict[el][1:])))
                       for el in neighbours_dict}
    neighbours_dict = {el: neighbours_dict[el][:15] for el in neighbours_dict}
    neighbours_dict = {ont + "#" + el: [ont + "#" + e for e in neighbours_dict[el]] for el in neighbours_dict}
    return neighbours_dict

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data(elem_tuple):
    op = np.array([[emb_indexer[el] for el in neighbours_dicts[elem.split("#")[0]][elem]] for elem in elem_tuple])
    return op

def generate_input(elems, target):
    inputs = np.array([generate_data(elem) for elem in list(elems)])
    targets = np.array([target for i in range(len(elems))])
    return inputs, targets

neighbours_dicts = {ont: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
max_neighbours = np.max(flatten([[len(el[e]) for e in el] for el in neighbours_dicts.values()]))
neighbours_lens = {ont: {key: len(neighbours_dicts[ont][key]) for key in neighbours_dicts[ont]}
                   for ont in neighbours_dicts}
neighbours_dicts = {ont: {key: neighbours_dicts[ont][key] + ["<UNK>" for i in range(max_neighbours -len(neighbours_dicts[ont][key]))]
              for key in neighbours_dicts[ont]} for ont in neighbours_dicts}

# data_items = data.items()
# np.random.shuffle(list(data_items))
# data = OrderedDict(data_items)

data = OrderedDict(sorted(data.items(), key=lambda x: x[0]))

print ("Number of entities:", len(data))
# all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))

all_metrics = []

for i in list(range(0, len(all_ont_pairs), 3)):
    
    test_onto = all_ont_pairs[i:i+3]
    
    train_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) not in test_onto}
    test_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) in test_onto}

    torch.set_default_dtype(torch.float64)
    
    train_test_split = 0.9

    train_data_t = [key for key in train_data if data[key]]
    train_data_f = [key for key in train_data if not data[key]][:len(train_data_t)]
#     [:int(0.1*(len(train_data) - len(train_data_t)) )]
#     np.random.shuffle(train_data_f)
    
    lr = 0.001
    num_epochs = 50
    weight_decay = 0.001
    batch_size = 10
    dropout = 0.3
    batch_size = min(batch_size, len(train_data_t))
    num_batches = int(ceil((len(train_data_t) + len(train_data_f))/batch_size))
    
    
    model = SiameseNetwork(512, 512, 1)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        inputs_pos, targets_pos = generate_input(train_data_t, 1)
        inputs_neg, targets_neg = generate_input(train_data_f, 0)
        
#         indices = np.random.permutation(len(inputs_pos) + len(inputs_neg))
        
#         inputs = np.array(list(inputs_pos) + list(inputs_neg))[indices]
#         targets = np.array(list(targets_pos) + list(targets_neg))[indices]

        inputs = np.array(list(inputs_pos) + list(inputs_neg))
        targets = np.array(list(targets_pos) + list(targets_neg))

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size
            
            inp = inputs[batch_start:batch_end].transpose(1,0,2)
#             print (inputs[batch_start:batch_end].resha.shape)
            nonzero_elems = np.count_nonzero(inp, axis=-1)
            indices = np.flip(np.argsort(nonzero_elems, axis=-1), axis=-1)
            seq_lens = np.flip(np.sort(nonzero_elems, axis=-1), axis=-1)
            inp_elems = np.stack((inp[0][[indices[0]]], inp[1][[indices[1]]]), axis=0)
            
            inp_elems = torch.LongTensor(inp_elems)
            targ_elems = torch.DoubleTensor(targets[batch_start:batch_end])
            optimizer.zero_grad()
            
            d1 = {elem:i for i,elem in enumerate(indices[0])}
            d2 = {elem:i for i,elem in enumerate(indices[1])}
            rev_indices = np.stack(([d1[k] for k in range(inp.shape[1])], 
                                    [d2[k] for k in range(inp.shape[1])]))

            rev_indices = torch.LongTensor(rev_indices)
            seq_lens = torch.LongTensor(seq_lens.copy())
#             print (inp_elems)
            outputs = model(inp_elems, seq_lens, rev_indices)
#             print ("outputs", outputs)
#             print ("targets", targ_elems)
            loss = F.mse_loss(outputs, targ_elems)
            loss.backward()
#             break
            optimizer.step()

            if batch_idx%10 == 0:
                print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

#         break
    model.eval()
#     break
    
    test_data_t = [key for key in test_data if data[key]]
    test_data_f = [key for key in test_data if not data[key]]
    
    res = greedy_matching()
    break
# print ("Final Results: ", np.mean([el[1] for el in all_metrics], axis=0))
# print ("Best threshold: ", all_metrics[np.argmax([el[1][2] for el in all_metrics])][0])

Number of entities: 122893


/home/vlead/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Epoch: 0 Idx: 0 Loss: 2.087074380623987e-05
Epoch: 0 Idx: 10 Loss: 1.3472349999682434e-08
Epoch: 0 Idx: 20 Loss: 1.3424823947934334e-13
Epoch: 0 Idx: 30 Loss: 0.9999999784640442
Epoch: 0 Idx: 40 Loss: 0.9999999708097326
Epoch: 0 Idx: 50 Loss: 0.9055666863689836
Epoch: 1 Idx: 0 Loss: 5.877148812401375e-09
Epoch: 1 Idx: 10 Loss: 2.5111411006464773e-14
Epoch: 1 Idx: 20 Loss: 4.675262043629766e-16
Epoch: 1 Idx: 30 Loss: 0.9999999839126194
Epoch: 1 Idx: 40 Loss: 0.9999999937796236
Epoch: 1 Idx: 50 Loss: 0.9999994438889962
Epoch: 2 Idx: 0 Loss: 2.924882980596831e-13
Epoch: 2 Idx: 10 Loss: 1.247556219719615e-14
Epoch: 2 Idx: 20 Loss: 1.1764164775961952e-16
Epoch: 2 Idx: 30 Loss: 0.9999999152870386


KeyboardInterrupt: 

In [52]:

def djikstra(n, adj_matrix, start_node): 

    distances = [sys.maxsize] * n 
    path = [False] * n
    
    distances[start_node] = 0
    
    for node in range(n): 
        
        distances_dict = {elem: i for (i,elem) in enumerate(distances) if not path[i]}
        closest_node = distances_dict[min(list(distances_dict.keys()))]

        path[closest_node] = True
 
        for curr_node in range(n): 
            if adj_matrix[closest_node][curr_node] > 0 and not path[curr_node] and \
            distances[curr_node] > distances[closest_node] + adj_matrix[closest_node][curr_node]: 
                distances[curr_node] = distances[closest_node] + adj_matrix[closest_node][curr_node]

    return distances

all_triples = Ontology("conference_ontologies/conference.owl").get_triples()

entities = {entity:i for i,entity in enumerate(list(set(flatten([(el[0], el[1]) for el in all_triples]))))}
entities_inv = {entities[entity]:entity for entity in entities}

adj_mat = np.zeros((len(entities), len(entities)))

for (a,b,_) in all_triples:
    adj_mat[entities[a]][entities[b]] = 1
    adj_mat[entities[b]][entities[a]] = 1

src = entities["Conference_part"]
sorted([(entities_inv[i], entity) for i,entity in enumerate(djikstra(len(entities), adj_mat, src))], key=lambda x:x[1])

[('Conference_part', 0),
 ('Workshop', 1.0),
 ('string', 1.0),
 ('Tutorial', 1.0),
 ('Topic', 1.0),
 ('Track', 1.0),
 ('Track-workshop_chair', 1.0),
 ('Conference_volume', 1.0),
 ('Program_committee', 2.0),
 ('Person', 2.0),
 ('Conference', 2.0),
 ('Organizing_committee', 2.0),
 ('Steering_committee', 2.0),
 ('Important_dates', 2.0),
 ('Review_preference', 2.0),
 ('Conference_www', 2.0),
 ('Conference_contribution', 2.0),
 ('Committee', 2.0),
 ('Conference_proceedings', 2.0),
 ('Publisher', 2.0),
 ('Committee_member', 3.0),
 ('Submitted_contribution', 3.0),
 ('Written_contribution', 3.0),
 ('date', 3.0),
 ('Conference_contributor', 3.0),
 ('Poster', 3.0),
 ('int', 3.0),
 ('Co-chair', 3.0),
 ('Conference_document', 3.0),
 ('Conference_participant', 3.0),
 ('Chair', 3.0),
 ('Conference_applicant', 3.0),
 ('Reviewer', 3.0),
 ('Presentation', 3.0),
 ('Thing', 3.0),
 ('Review_expertise', 4.0),
 ('Invited_speaker', 4.0),
 ('Reviewed_contribution', 4.0),
 ('Invited_talk', 4.0),
 ('Registeered

In [46]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

['Workshop',
 'string',
 'Tutorial',
 'Topic',
 'Track',
 'Track-workshop_chair',
 'Conference_volume']

In [21]:
emb_indexer_inv[135].

(array([[702, 777, 702, 754, 702,   2, 702, 597, 497, 451],
        [398, 457, 135, 409, 135, 582, 135, 131, 135, 438]]),
 array([[[702],
         [777]],
 
        [[702],
         [754]],
 
        [[702],
         [  2]],
 
        [[702],
         [597]],
 
        [[497],
         [451]],
 
        [[398],
         [457]],
 
        [[135],
         [409]],
 
        [[135],
         [582]],
 
        [[135],
         [131]],
 
        [[135],
         [438]]]))

In [15]:
import pickle
f = open("embeddings.pkl", "rb")
embeddings, all_ont_pairs = pickle.load(f)